# Tensorflow Estimator

In [1]:
import tensorflow as tf

import pandas as pd

import numpy as np 

from sklearn.model_selection import train_test_split


wines_df = pd.read_csv("../data/winequality.csv")



# tf doesn't like spaces in col names so I replace them with _ 

new_col_list = []
for col_name in wines_df.columns:
    new_col_names = col_name.replace(" ", "_")
    new_col_list.append(new_col_names)
wines_df.columns = new_col_list


# Change the type of the index values from [0....1598] to [wine_1....wine_1599]

wines_df.index += 1 # add 1 to index values to start wine specification from wine_1 rather than wine_0

index_as_string = wines_df.index.astype('str')

wines_df.index = 'wine_' + index_as_string


## Splitting the dataset to training, validation and test sets

In [2]:
# shuffle the data of the wines_df

wines_df = wines_df.sample(frac=1) 

# almost 70% training , 15% validation, 15% test set

intermediate_set, valid_set = train_test_split(wines_df, test_size=0.15) 
train_set, test_set = train_test_split(intermediate_set, test_size=0.15)

In [ ]:
display(train_set)

In [4]:
train_y = train_set.pop('quality')

In [ ]:
display(train_y)

In [ ]:
train_set has 1155 examples
for batch size equal to 33 we have batch_count = 1155/33 = 35




In [ ]:
1155/33

## Input functions

In [ ]:
# creating the input function for training the estimator with feature columns

def feature_columns_train_input_func(dataframe=train_set, batch_size=33):
    
    
    dataframe = dict(dataframe) 
    
    label_srs = dataframe.pop('quality')
    
    label_dict = {}
    
    label_dict['quality'] = label_srs
    
    
    # normalization
    for name in dataframe.keys():
        print(name)
        min_value = dataframe[name].min()
        max_value = dataframe[name].max()
        dataframe[name] = (dataframe[name] - min_value)/(max_value-min_value) 
    #
    
    
    dataset = tf.data.Dataset.from_tensor_slices((dataframe, label_dict))
    
    dataset = dataset.batch(batch_size)
    
    
    
        
            
    return dataset



In [5]:
# creating the input function for training the estimator

def no_feat_train_input_func(dataframe=train_set, labels = train_y, batch_size=33):
    
    
    
    
    
    
    
    # normalization
    for name in dataframe.columns:
        min_value = dataframe[name].min()
        max_value = dataframe[name].max()
        dataframe[name] = (dataframe[name] - min_value)/(max_value-min_value) 
    #
    
    #dataframe = dataframe.iloc[1:,]
    
    
    dataset = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    

    dataset = dataset.batch(batch_size)
    
    
    return dataset



In [ ]:
def eval_input_func(dataframe=test_set, batch_size=33):
    
    
    dataframe = dataframe.copy() 
    
    label_values = dataframe.pop('quality')
    
    
    # normalization
    for name in dataframe.columns:
        min_value = dataframe[name].min()
        max_value = dataframe[name].max()
        dataframe[name] = (dataframe[name] - min_value)/(max_value-min_value) 
    #

    dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), label_values))
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [ ]:
def serving_input_func():
    pass

In [6]:
data = no_feat_train_input_func()

/home/jpriest/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [82]:
a = tf.constant([[1,2,3],[4,5,6]])


print(tf.shape(a))



tf.Tensor([2 3], shape=(2,), dtype=int32)


In [101]:
# Doulepse mexri ena shmeio!
for row in data:

    feat , labels = row

    #architecture
    dense_1 = tf.keras.layers.Dense(units=2, input_shape=(33,11), activation='relu', dtype='float32')
    dense_2 = tf.keras.layers.Dense(units=10, input_shape=(33,2), activation="softmax", dtype='float32')
    
    
    # Trainable Variables for weights and biases
    
    #w1 = tf.Variable(tf.random.normal(shape=(11,2)), trainable=True, shape=(11,2), name='d1_weights')
    #b1 = tf.Variable(tf.random.normal(shape=(2,)), trainable=True, shape=(2,), name = 'd1_bias')
    #w2 = tf.Variable(tf.random.normal(shape=(2,10)), trainable=True, shape=(2,10), name = 'd2_weights')
    #b2 = tf.Variable(tf.random.normal(shape=(10,)), trainable = True, shape = (10,), name = 'd2_bias')
    
    
    
    
    
    
    #model_output tensors
    
        
    d1_out = dense_1(feat)
        
    probs = dense_2(h1_layer)
    
    w1 = dense_1.trainable_weights[0]
    b1 = dense_1.trainable_weights[1]
    w2 = dense_2.trainable_weights[0]
    b2 = dense_2.trainable_weights[1]
    
    my_var_list = [w1,b1,w2,b2]
    
    
    
    
    class_predictions = tf.argmax(probs, axis=1, output_type=tf.int64)

        # check the first data point
    print(f"The probabilities of each class in the first data point are {prob_layer[0,:]}")
    print(f"The index of the most probable class for the first data point is {class_predictions[0]}")

    
    loss = tf.keras.losses.sparse_categorical_crossentropy(labels, probs)
    
    
    
    print("######")
    
    print("The loss is {}".format(loss))
    print(loss.shape)
    
    # xrhsh optimizer
    
    adam = tf.keras.optimizers.Adam()
    train_op = adam.minimize(loss = lambda: loss, var_list=my_var_list)
    
    print(train_op)
    
    break

The probabilities of each class in the first data point are [0.11179995 0.09663981 0.07063343 0.0647497  0.07776936 0.14152844
 0.1288863  0.09249505 0.0783148  0.13718322]
The index of the most probable class for the first data point is 1
######
The loss is [2.266869  2.2074332 2.2138152 2.2320557 2.2212014 2.195403  2.4483783
 2.2744503 2.2136328 2.212556  2.267128  2.2674255 2.2240348 2.2227445
 2.203975  2.205077  2.2682652 2.2092562 2.222239  2.006835  2.2066283
 2.7156682 2.2668116 2.2700124 2.200361  2.196082  1.9207166 2.396748
 2.2081294 2.2688355 2.2728403 2.3845756 2.2683618]
(33,)


ValueError: No gradients provided for any variable: ['dense_136/kernel:0', 'dense_136/bias:0', 'dense_137/kernel:0', 'dense_137/bias:0'].

## Feature Columns

In [71]:
# Specifying the types of feature columns in the dataset
feature_columns = []

for features in train_set.columns:
    
    if features != 'quality':
        
         numeric_columns.append(tf.feature_column.numeric_column(features))
        # i get the batch size here so it can be used in the input layer!
        # I represent each feature as a feature block with shape 0 or (1,)? features are scalar values
        #feature_input_layer_block[features] = tf.keras.Input(shape=(1,), batch_size=33,   name=features)
        


In [ ]:
train_data = train_input_func()

{'column_names': ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']}


## Model Function

### Declaration and modes

In [43]:
# model_fn without feature_columns

def my_second_model_fn(features, labels, mode, params):
    
    
    # constracting a dense feature layer blueprint based on the type of feature columns
    '''
    
    '''
    
   
    
    dense_1 = tf.keras.layers.Dense(units=20, input_shape=(33,11), activation='relu', dtype='float32')
    dense_2 = tf.keras.layers.Dense(units=10, input_shape=(33,20), activation='softmax', dtype='float32')
    
    
    
    #model
    
    d1_output = dense_1(features)
    probs = dense_2(d1_output)
    
    #print(probs.trainable_weights)
    
    # Grab Weights as trainable tf.Variables
    
    dense_1_weights = dense_1.trainable_weights
    dense_2_weights = dense_2.trainable_weights
    
    var_list = list()
    var_list.append(dense_1_weights[0])
    var_list.append(dense_1_weights[1])
    var_list.append(dense_2_weights[0])
    var_list.append(dense_2_weights[1])
    
    

    if mode == tf.estimator.ModeKeys.TRAIN:

        loss = tf.keras.losses.sparse_categorical_crossentropy(labels,probs)
        optimizer = tf.keras.optimizers.Adam()
        train_op = optimizer.minimize(lambda:loss , lambda:var_list)
        predictions = None
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

    elif mode == tf.estimator.ModeKeys.EVAL:

        loss = tf.keras.losses.sparse_categorical_crossentropy()
        metrics = tf.keras.metrics.Accuracy()
        train_op = None
        predictions = None

    elif mode == tf.estimator.ModeKeys.PREDICT:


        loss = None
        train_op = None
        predictions = {
            'predicted_class' : tf.argmax(logits),
            'probabilities' : tf.nn.softmax(logits)
        }



In [ ]:
# feature columns model_fn

def my_model_fn(features, labels, mode, params):
    
    
    
    # constracting a dense feature layer blueprint based on the type of feature columns
    '''
    pws pairnw features kai labels?

    o alex xrhsimopoiei ena function token map pad 
    '''
  
   ### I want lines 20 to 41 outside of the model_fn
    
    
    
    feature_columns = []
    feature_layer_inputs = {}
    
    
    for feature in train_set.columns: # problem here because I need the train_set to be passed as parameters
    
        if feature != 'quality':
        
            feature_columns.append(tf.feature_column.numeric_column(feature))
            feature_layer_inputs[feature] = tf.keras.Input(dtype=tf.dtypes.float32,shape=(), batch_size=33,   name=feature)
        
    
    
    
    # use feature columns in model_fn
    
    
    
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    
    #print(tf.shape(feature_layer))
    
    
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    #print(feature_layer_outputs)
    #
    
    h1 = tf.keras.layers.Dense(units=128, activation='relu', dtype='float32')(feature_layer_outputs)
    
    print(h1.shape)
    
    h2 = tf.keras.layers.Dense(units=64,   activation = 'relu', dtype='float32')(h1)
    
    print(h2.shape)
    
    probs = tf.keras.layers.Dense(units=10, activation='softmax', dtype='float32')(h2)
    
    print(probs)
    
    
    y_pred = tf.math.argmax(input=probs,  output_type=tf.dtypes.int32, name='predictions')
    
    
    
    print("#### :) ####")
    
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        print("hi")
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels,probs)
        print(my_loss)
        optimizer = tf.keras.optimizers.Adam()
        train_op = optimizer.minimize(loss)
        predictions = None

    elif mode == tf.estimator.ModeKeys.EVAL:

        loss = tf.keras.losses.sparse_categorical_crossentropy()
        metrics = tf.keras.metrics.Accuracy()
        train_op = None
        predictions = None

    elif mode == tf.estimator.ModeKeys.PREDICT:


        loss = None
        train_op = None
        predictions = {
            'predicted_class' : tf.argmax(logits),
            'probabilities' : tf.nn.softmax(logits)
        }



In [44]:
my_estimator = tf.estimator.Estimator(model_fn = my_second_model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpltfx38je', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
my_estimator.train(input_fn=lambda:no_feat_train_input_func(train_set, train_y))

INFO:tensorflow:Calling model_fn.
<tf.Variable 'dense/kernel:0' shape=(11, 20) dtype=float32>
 
 
 
<tf.Variable 'dense/bias:0' shape=(20,) dtype=float32>


/home/jpriest/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


ValueError: No gradients provided for any variable: ['dense/kernel:0', 'dense/bias:0', 'dense_1/kernel:0', 'dense_1/bias:0'].